This code pulls variables from multiple clean and processed csvs to create the dataframes used in the behavioral analyses.

In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()

'/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement'

In [6]:
# read in data
meta = pd.read_csv('clean-data/metadata.csv')
beh = pd.read_csv('clean-data/first_clips_clean.csv')
flight = pd.read_csv('processed-data/drone_variables_flight.csv')
clip = pd.read_csv('processed-data/drone_variables_clip2.csv')
env = pd.read_csv('processed-data/env_variables_clip2.csv')
ground = pd.read_csv('clean-data/ground_conditions.csv')

In [12]:
# Gather all columns in one dataframe, matching up by flight and clip_type
new_df = pd.merge(meta, beh, on = 'flight', how = 'outer')
new_df = pd.merge(new_df, flight, on = 'flight', how = 'outer')
new_df = pd.merge(new_df, clip, on = ['flight'], how = 'outer')
new_df = pd.merge(new_df, env, on = ['flight'], how = 'outer')
new_df = pd.merge(new_df, ground, on = 'flight', how = 'outer')
new_df.replace('NAN', np.nan, inplace = True)

In [ ]:
# add in flight type (initial or replacement)
new_df['flight_type'] = ['initial' if '-01' in i else 'replacement' for i in new_df['flight']]

In [10]:
beh

,observation,flight,site,species,group_size_spp,group_size_total,young_present,note,exclude,group_behavior
0,ob001,001-01,opc,gg,5.0,21.0,False,NaN,NaN,2.0
1,ob001,001-01,opc,tg,16.0,21.0,True,NaN,NaN,1.0
2,ob002,002-01,opc,buf,14.0,14.0,False,NaN,NaN,1.0
3,ob003,003-01,opc,tg,5.0,26.0,True,NaN,NaN,NaN
4,ob003,003-01,opc,pz,21.0,26.0,True,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
149,ob114,114-01,mpala,gz,2.0,23.0,False,NaN,NaN,0.0
150,ob114,114-01,mpala,imp,21.0,23.0,True,NaN,NaN,0.0
151,ob115,115-01,mpala,gz,1.0,30.0,False,NaN,NaN,0.0
152,ob115,115-01,mpala,pz,19.0,30.0,True,NaN,NaN,0.0
